In [ ]:
%load_ext autoreload
%autoreload 2
import datajoint as dj
import numpy as np
import pandas as pd
from pipeline.ingest import behavior as behavior_ingest
from pipeline.ingest import ephys as ephys_ingest
from pipeline import lab, get_schema_name, experiment, foraging_analysis, report, ephys
import pipeline.shell as shell
shell.logsetup('INFO')
[hh for hh in dj.list_schemas() if 'hanhou' in hh]

# Migrating my code

# Show schema

In [ ]:
dj.ERD(foraging_analysis) + dj.ERD(lab) + dj.ERD(experiment) 

In [ ]:
lab.WaterRestriction()

In [ ]:
dj.ERD(ephys)

In [ ]:
dj.ERD(ephys_ingest)

## Ingest meta data

In [ ]:
shell.load_meta_foraging()

In [ ]:
(lab.WaterRestriction() * lab.Subject.proj(cage_number_subject="cage_number")).fetch(format='frame').sort_values(by='water_restriction_number')

In [ ]:
experiment.SessionComment()

## Ingest foraging sessions

In [ ]:
shell.ingest_foraging_behavior()

In [ ]:
behavior_ingest.BehaviorBpodIngest()

In [ ]:
experiment.Session()

# Query testing

In [ ]:
lab.Subject().

In [ ]:
lab.WaterRestriction().fetch()

In [ ]:
lab.WaterRestriction & behavior_ingest.BehaviorBpodIngest()

In [ ]:
(experiment.Session  * experiment.SessionComment * experiment.SessionDetails & 'subject_id = 455525').fetch()[-1]

In [ ]:
experiment.TaskProtocol()

In [ ]:
experiment.TrialNote() * lab.WaterRestriction & 'trial_note_type="random_seed_start"'

In [ ]:
experiment.SessionComment()

In [ ]:
experiment.SessionTask()

In [ ]:
(lab.WaterRestriction() & (experiment.BehaviorTrial() & 'task in ("foraging", "foraging 3lp")')).fetch(format='frame').sort_values(by='water_restriction_number')

Number of foraging sessions for each mice

In [ ]:
foraging_sessions = (lab.WaterRestriction() * experiment.Session & 
                    (experiment.BehaviorTrial & 'task in ("foraging", "foraging 3lp")'))\
                    .fetch(format='frame')
foraging_sessions.groupby(['water_restriction_number', 'subject_id']).size()

In [ ]:
len(foraging_sessions)

Two-lickport only

In [ ]:
foraging_sessions = (lab.WaterRestriction() * experiment.Session & 
                    (experiment.BehaviorTrial & 'task in ("foraging")'))\
                    .fetch(format='frame')
foraging_sessions.groupby(['water_restriction_number', 'subject_id']).size()

In [ ]:
(lab.WaterRestriction() * experiment.Session &
                    (experiment.BehaviorTrial & 'task in ("foraging")')) & 'water_restriction_number="FOR05"'

In [ ]:
(experiment.Session() & (lab.WaterRestriction() & 'water_restriction_number="HH01"')).fetch(format='frame')

In [ ]:
HC16_sessions = experiment.Session() & (lab.WaterRestriction() & 'water_restriction_number="HC16"')

In [ ]:
HC16_sessions

In [ ]:
temp_session = (experiment.BehaviorTrial() * experiment.WaterPortChoice() & HC16_sessions & 'session=2')
temp_session.fetch(format='frame')

Check block info

In [ ]:
(experiment.SessionBlock & temp_session).WaterPortRewardProbability() * ((experiment.SessionBlock & temp_session).BlockTrial())

In [ ]:
temp_session.fetch(format='frame')[['water_port', 'outcome']].drop_duplicates()

In [ ]:
temp_session.fetch(format='frame')['outcome'].value_counts()

# Populate Behavior_foraging for inspector

In [ ]:
dj.ERD(foraging_analysis)

In [ ]:
dj.ERD(experiment.BehaviorTrial) -1 +1

Individual testimDatPrb_port=1


In [ ]:
foraging_analysis.TrialReactionTime.populate(display_progress=True, reserve_jobs=False)

In [ ]:
foraging_analysis.BlockStats.populate(display_progress=True, reserve_jobs=False)

In [ ]:
foraging_analysis.SessionStats.populate(display_progress=True, reserve_jobs=False)

In [ ]:
foraging_analysis.SessionStats()

In [ ]:
foraging_analysis.SessionTaskProtocol.populate(display_progress=True, reserve_jobs=False)

In [ ]:
foraging_analysis.BlockFraction.populate(display_progress=True, reserve_jobs=False)

In [ ]:
behavior_foraging.SessionMatching.populate(display_progress=True, reserve_jobs=False)

In [ ]:
foraging_analysis.BlockEfficiency.populate(display_progress=True, reserve_jobs=False)

Populate using parallel processing

Show progress

In [ ]:
schema = dj.schema(get_schema_name('foraging_analysis'),locals())
schema.jobs

In [ ]:
schema

In [ ]:
schema.jobs.fetch('key')

In [ ]:
schema.jobs.delete() 

In [ ]:
dj.kill()

Check progress

In [ ]:
length = len(foraging_analysis.TrialStats())
ratio = length / len(experiment.BehaviorTrial())
print(f'TrialReactiontime:{length}, {ratio:.2%}')

In [ ]:
length = len(foraging_analysis.BlockEfficiency())
ratio = length / len(foraging_analysis.BlockFraction())
print(f'TrialReactiontime:{length}, {ratio:.2%}')

In [ ]:
foraging_analysis.BlockEfficiency.drop()

In [ ]:
length = len(report.SessionLevelForagingSummary())
ratio = length / len(experiment.Session()&(experiment.BehaviorTrial & 'task = "foraging"'))
print(f'SessionLevelForagingSummary:{length}, {ratio:.2%}')

In [ ]:
length = len(report.SessionLevelForagingLickingPSTH())
ratio = length / len(experiment.Session()&(experiment.BehaviorTrial & 'task = "foraging"'))
print(f'SessionLevelForagingLickingPSTH:{length}, {ratio:.2%}')

# Plot training summary

In [ ]:
foraging_analysis.SessionStats()

In [ ]:
(experiment.BehaviorTrial  & 'outcome = "hit"')

In [ ]:
(foraging_analysis.SessionStats & (lab.WaterRestriction & 'water_restriction_number="HH07"')).fetch(format='frame')

In [ ]:
from pipeline.plot import foraging_plot
foraging_plot.plot_training_summary()

# Clear all (Nuclear)

In [ ]:
schema_to_clear = ['hanhou_foraging_ingest_behavior', 'hanhou_foraging_experiment', 'hanhou_foraging_lab', 'hanhou_foraging_ccf']
for ss in schema_to_clear:
    dj.schema(ss).drop()

# Clear Behavior_foraging only

In [ ]:
schema_to_clear = ['hanhou_foraging_foraging_analysis']
for ss in schema_to_clear:
    dj.schema(ss).drop()

# Drop tables on the server

In [ ]:
schema = dj.schema(get_schema_name('foraging_analysis'))
schema.spawn_missing_classes()
dj.ERD(schema)

In [ ]:
SessionMatchBias.drop()

# Crap

In [ ]:
schema = dj.schema('hanhou_test')

In [ ]:
dj.ERD(schema)

In [ ]:
@schema
class Test(dj.Manual):
    definition = """
    # Comment for table
    id: int  # id
    ---
    # this is a comment
    label = NULL: int   # Real comment  
    # another comment
    label2 = NULL: int
    """
    
@schema
class Test2(dj.Manual):
    definition = """
    # Comment for table
    id: int  # id
    -> Test
    ---
    # this is a comment
    label = NULL: int   # Real comment  
    # another comment
    label2 = NULL: int
    """

In [ ]:
Test2.drop()

In [ ]:
toinsert = dict(id=3, label2=2)
Test.insert1(toinsert)

In [ ]:
dj.ERD(schema)
Test()